In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [2]:
path = 'combined_collisions.csv'

df = pd.read_csv(path)
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [4]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

##### Below selects out region. You can use Dan's feature to select Greater London, this selects Central London.
- additionally I drop all entries with no traffic counter matches

In [6]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

df_lond = df.loc[I,:].copy()

In [7]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Make neighborhoods feature

In [8]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

#### Make a DataFrame using df_lond that has entries only for traffic IDs. Main feature is # of incidents.¶
- Other features are the aadf features that go along with the ID
- Also maybe some averaged features from accidents dataset (like junction control, maybe just use the classification with the most entries for that ID)
- Maybe the target variable should be number of incidents normalized by the bike volume (this is prob. of an incident)

##### Features to include
- all aadf features (do they neeed to be averaged over year? Maybe just average anyway)
- modes of classification features in accidents
- num. of incidents
- num. of serious and fatal incidents
- num. of incidents normalized to bike volume from aadf

In [10]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

Nthresh = 10 #this sets which traffic counters are included, they must have at least this many accidents 

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['aadf_Road_name','aadf_Road_category','aadf_Road_type']
for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
df_road['total_norm'] = df_road['total'].values/df_road['aadf_Pedal_cycles'].values
df_road['serious_ratio'] = df_road['serious'].values/df_road['total'].values

In [11]:
#add in aadf year availability metadata 
path_year = 'aadf_year_meta.csv'
df_year = pd.read_csv(path_year)

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nid).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

#### Here is the final DataFrame

In [13]:
df_road.sample(20)

,ID,total,slight,serious,neigh,Road_name,Road_name2,h,Day_of_Week,m,...,aadf_All_motor_vehicles,aadf_All_HGVs,aadf_LGVs,aadf_Buses_and_coaches,total_norm,serious_ratio,N_year,Year_start,Year_end,Max_year_gap
130,16621,35,33,2,15,A107,U,17,4,7,...,20156.285714,922.114286,2852.142857,2235.428571,0.042069,0.057143,22,2000,2021,1
104,46268,43,33,10,2,A23,U,17,4,10,...,18188.813953,598.697674,2967.883721,1443.511628,0.011452,0.232558,22,2000,2021,1
29,36076,79,74,5,14,A1,U,9,4,4,...,24028.772152,916.405063,3641.721519,3012.379747,0.016506,0.063291,22,2000,2021,1
139,36218,33,28,5,10,A13,U,17,6,6,...,16658.636364,980.272727,2439.727273,763.545455,0.083404,0.151515,22,2000,2021,1
331,38555,10,9,1,12,A4201,A5205,9,7,4,...,20577.700000,791.900000,2952.100000,578.900000,0.004196,0.100000,22,2000,2021,1
243,73680,19,16,3,8,A301,A301,12,5,6,...,19268.684211,552.210526,2393.157895,3304.473684,0.005032,0.157895,22,2000,2021,1
314,57657,12,10,2,2,A3204,U,7,2,1,...,18373.916667,797.833333,3950.000000,304.166667,0.022613,0.166667,22,2000,2021,1
9,16188,130,109,21,9,A11,U,18,3,9,...,21053.084615,1337.107692,2930.646154,1542.576923,0.078163,0.161538,22,2000,2021,1
177,47727,26,24,2,13,A4201,U,18,4,6,...,15846.115385,461.153846,2534.192308,379.923077,0.020266,0.076923,22,2000,2021,1
59,48161,57,48,9,9,A1202,U,9,2,8,...,28364.210526,2124.894737,5512.192982,278.140351,0.022294,0.157895,22,2000,2021,1
